In [1]:
# this converts a zst file to csv
#
# it's important to note that the resulting file will likely be quite large
# and you probably won't be able to open it in excel or another csv reader
#
# arguments are inputfile, outputfile, fields
# call this like
# python to_csv.py wallstreetbets_submissions.zst wallstreetbets_submissions.csv author,selftext,title

import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers
import pandas as pd


log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()


if __name__ == "__main__":
	input_file_path = "relationships_comments_new.zst"
	output_file_path = "relationships_comments_17_22.csv"
	fields = "author"

	file_size = os.stat(input_file_path).st_size
	file_lines = 0
	file_bytes_processed = 0
	line = None
	created = None
	bad_lines = 0
	output_file = open(output_file_path, "w", encoding='utf-8', newline="")
	writer = csv.writer(output_file)
	#writer.writerow(fields)
	rows=[]
	try:
		for line, file_bytes_processed in read_lines_zst(input_file_path):
			try:
				obj = json.loads(line)
				output_obj = []
				#print(obj)
				#rows.append(obj)
                
				#for field in fields:
					#output_obj.append(str(obj[field]).encode("utf-8", errors='replace').decode())
				#writer.writerow(output_obj)
#first date is end date und last date is first date
				created = datetime.utcfromtimestamp(int(obj['created_utc']))
				if created.strftime('%Y-%m-%d %H:%M:%S') < "2016-11-30 23:59:59" and created.strftime('%Y-%m-%d %H:%M:%S')  > "2016-11-01 00:00:00":
					rows.append(obj)
			except json.JSONDecodeError as err:
				bad_lines += 1
			file_lines += 1
			#if file_lines % 100000 == 0:
				#log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
	except KeyError as err:
		log.info(f"Object has no key: {err}")
		log.info(line)
	except Exception as err:
		log.info(err)
		log.info(line)

	output_file.close()
	log.info(f"Complete : {file_lines:,} : {bad_lines:,}")
	df=pd.DataFrame.from_dict(rows, orient='columns')

Complete : 26,908,630 : 0


In [2]:
df

,author_flair_css_class,subreddit_id,author,author_flair_text,score,subreddit,controversiality,stickied,link_id,body,distinguished,gilded,edited,id,retrieved_on,created_utc,parent_id
0,None,t5_2qjvn,antioch75,None,4,relationships,0,False,t3_5aevs2,You really need to get a run down of what actu...,None,0,False,d9g52zy,1481097670,1477958423,t3_5aevs2
1,None,t5_2qjvn,screelov,None,1,relationships,0,False,t3_5ack9c,"Get out of there, and don't look back. Don't s...",None,0,False,d9g532l,1481097671,1477958426,t3_5ack9c
2,None,t5_2qjvn,loves_small_motors,None,294,relationships,0,False,t3_5afv9z,"And honestly, it's pretty bizarre that someone...",None,0,False,d9g53m1,1481097678,1477958450,t1_d9g4b2o
3,None,t5_2qjvn,[deleted],None,-5,relationships,0,False,t3_5aejxb,[removed],None,0,False,d9g53o3,1481097679,1477958452,t1_d9g0ai7
4,None,t5_2qjvn,[deleted],None,1,relationships,0,False,t3_5afuv9,[removed],None,0,False,d9g53rk,1481097680,1477958456,t1_d9g518n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263888,None,t5_2qjvn,slinky999,None,16,relationships,0,False,t3_5ft3gn,I would also add that I was desperately unhapp...,None,0,False,damuewi,1481891691,1480550378,t1_damu47z
263889,None,t5_2qjvn,shelbyknits,None,1,relationships,0,False,t3_5ft6ew,"Given that she's been having loud, personal ca...",None,0,False,damueyo,1481891692,1480550380,t3_5ft6ew
263890,None,t5_2qjvn,theonewithoutapic,None,13,relationships,0,False,t3_5friad,"""Exercise gives you endorphins. Endorphins mak...",None,0,False,damuf0f,1481891692,1480550382,t1_damhkav
263891,None,t5_2qjvn,YouKnowYourCrazy,None,1,relationships,0,False,t3_5fsuzp,Context is everything. If you put this in the ...,None,0,False,damuff2,1481891699,1480550397,t1_damu86x


In [3]:
# this converts a zst file to csv
#
# it's important to note that the resulting file will likely be quite large
# and you probably won't be able to open it in excel or another csv reader
#
# arguments are inputfile, outputfile, fields
# call this like
# python to_csv.py wallstreetbets_submissions.zst wallstreetbets_submissions.csv author,selftext,title

import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers
import pandas as pd


log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()


if __name__ == "__main__":
	input_file_path = "relationships_comments_new.zst"
	output_file_path = "relationships_comments_17_22.csv"
	fields = "author"

	file_size = os.stat(input_file_path).st_size
	file_lines = 0
	file_bytes_processed = 0
	line = None
	created = None
	bad_lines = 0
	output_file = open(output_file_path, "w", encoding='utf-8', newline="")
	writer = csv.writer(output_file)
	#writer.writerow(fields)
	rows=[]
	try:
		for line, file_bytes_processed in read_lines_zst(input_file_path):
			try:
				obj = json.loads(line)
				output_obj = []
				#print(obj)
				#rows.append(obj)
                
				#for field in fields:
					#output_obj.append(str(obj[field]).encode("utf-8", errors='replace').decode())
				#writer.writerow(output_obj)
#first date is end date und last date is first date
				created = datetime.utcfromtimestamp(int(obj['created_utc']))
				if created.strftime('%Y-%m-%d %H:%M:%S') < "2017-11-30 23:59:59" and created.strftime('%Y-%m-%d %H:%M:%S')  > "2017-11-01 00:00:00":
					rows.append(obj)
			except json.JSONDecodeError as err:
				bad_lines += 1
			file_lines += 1
			#if file_lines % 100000 == 0:
				#log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
	except KeyError as err:
		log.info(f"Object has no key: {err}")
		log.info(line)
	except Exception as err:
		log.info(err)
		log.info(line)

	output_file.close()
	log.info(f"Complete : {file_lines:,} : {bad_lines:,}")
	df_2=pd.DataFrame.from_dict(rows, orient='columns')

Complete : 26,908,630 : 0
Complete : 26,908,630 : 0


In [4]:
df_2

,author,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,gilded,...,retrieved_on,score,stickied,subreddit,subreddit_id,subreddit_type,author_cakeday,mod_note,mod_reason_by,mod_reason_title
0,shelbyknits,None,None,"This is so true, and I’ve seen this even with ...",True,0,1509494401,None,False,0,...,1511998504,6,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
1,[deleted],None,None,[removed],True,0,1509494402,None,False,0,...,1511998504,1,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
2,UsagiDreams,None,None,Well...at least now you know how she's going t...,True,0,1509494402,None,False,0,...,1511998505,41,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
3,Sykryk,None,None,"Do what I did in these situations:\n\n""Were ha...",True,0,1509494404,None,False,0,...,1511998507,-1,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
4,Amairch,None,None,Well sounds like your wife needs to learn some...,True,1,1509494404,None,False,0,...,1511998507,0,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317165,Meraca,None,None,I agree with everyone here. Tell your mom. Wha...,True,0,1512086373,None,False,0,...,1513179510,2,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
317166,buttfacedmiscreant11,None,None,As someone who has lived with too many gross f...,True,0,1512086376,None,False,0,...,1513179511,2,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
317167,Mr_Bulldops33,None,None,That’s not a cougar it’s a damn predator who n...,True,0,1512086379,None,False,0,...,1513179512,0,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
317168,Seshology,None,None,I used to struggle with lying when I was a tee...,True,0,1512086389,None,False,0,...,1513179516,3,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN


In [5]:
df_2

,author,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,gilded,...,retrieved_on,score,stickied,subreddit,subreddit_id,subreddit_type,author_cakeday,mod_note,mod_reason_by,mod_reason_title
0,shelbyknits,None,None,"This is so true, and I’ve seen this even with ...",True,0,1509494401,None,False,0,...,1511998504,6,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
1,[deleted],None,None,[removed],True,0,1509494402,None,False,0,...,1511998504,1,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
2,UsagiDreams,None,None,Well...at least now you know how she's going t...,True,0,1509494402,None,False,0,...,1511998505,41,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
3,Sykryk,None,None,"Do what I did in these situations:\n\n""Were ha...",True,0,1509494404,None,False,0,...,1511998507,-1,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
4,Amairch,None,None,Well sounds like your wife needs to learn some...,True,1,1509494404,None,False,0,...,1511998507,0,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317165,Meraca,None,None,I agree with everyone here. Tell your mom. Wha...,True,0,1512086373,None,False,0,...,1513179510,2,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
317166,buttfacedmiscreant11,None,None,As someone who has lived with too many gross f...,True,0,1512086376,None,False,0,...,1513179511,2,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
317167,Mr_Bulldops33,None,None,That’s not a cougar it’s a damn predator who n...,True,0,1512086379,None,False,0,...,1513179512,0,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN
317168,Seshology,None,None,I used to struggle with lying when I was a tee...,True,0,1512086389,None,False,0,...,1513179516,3,False,relationships,t5_2qjvn,public,NaN,NaN,NaN,NaN


In [6]:
df_2.columns

Index(['author', 'author_flair_css_class', 'author_flair_text', 'body',
       'can_gild', 'controversiality', 'created_utc', 'distinguished',
       'edited', 'gilded', 'id', 'is_submitter', 'link_id', 'parent_id',
       'permalink', 'retrieved_on', 'score', 'stickied', 'subreddit',
       'subreddit_id', 'subreddit_type', 'author_cakeday', 'mod_note',
       'mod_reason_by', 'mod_reason_title'],
      dtype='object')

In [7]:
# this converts a zst file to csv
#
# it's important to note that the resulting file will likely be quite large
# and you probably won't be able to open it in excel or another csv reader
#
# arguments are inputfile, outputfile, fields
# call this like
# python to_csv.py wallstreetbets_submissions.zst wallstreetbets_submissions.csv author,selftext,title

import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers
import pandas as pd


log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()


if __name__ == "__main__":
	input_file_path = "relationships_comments_new.zst"
	output_file_path = "relationships_comments_17_22.csv"
	fields = "author"

	file_size = os.stat(input_file_path).st_size
	file_lines = 0
	file_bytes_processed = 0
	line = None
	created = None
	bad_lines = 0
	output_file = open(output_file_path, "w", encoding='utf-8', newline="")
	writer = csv.writer(output_file)
	#writer.writerow(fields)
	rows=[]
	try:
		for line, file_bytes_processed in read_lines_zst(input_file_path):
			try:
				obj = json.loads(line)
				output_obj = []
				#print(obj)
				#rows.append(obj)
                
				#for field in fields:
					#output_obj.append(str(obj[field]).encode("utf-8", errors='replace').decode())
				#writer.writerow(output_obj)
#first date is end date und last date is first date
				created = datetime.utcfromtimestamp(int(obj['created_utc']))
				if created.strftime('%Y-%m-%d %H:%M:%S') < "2018-11-30 23:59:59" and created.strftime('%Y-%m-%d %H:%M:%S')  > "2018-11-01 00:00:00":
					rows.append(obj)
			except json.JSONDecodeError as err:
				bad_lines += 1
			file_lines += 1
			#if file_lines % 100000 == 0:
				#log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
	except KeyError as err:
		log.info(f"Object has no key: {err}")
		log.info(line)
	except Exception as err:
		log.info(err)
		log.info(line)

	output_file.close()
	log.info(f"Complete : {file_lines:,} : {bad_lines:,}")
	df_3=pd.DataFrame.from_dict(rows, orient='columns')

Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0


In [8]:
df_3.columns

Index(['archived', 'author', 'author_created_utc',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_template_id',
       'author_flair_text', 'author_flair_text_color', 'author_flair_type',
       'author_fullname', 'author_patreon_flair', 'body', 'can_gild',
       'can_mod_post', 'collapsed', 'collapsed_reason', 'controversiality',
       'created_utc', 'distinguished', 'edited', 'gilded', 'gildings', 'id',
       'is_submitter', 'link_id', 'no_follow', 'parent_id', 'permalink',
       'removal_reason', 'retrieved_on', 'score', 'send_replies', 'stickied',
       'subreddit', 'subreddit_id', 'subreddit_name_prefixed',
       'subreddit_type', 'author_cakeday'],
      dtype='object')

In [9]:
df_3

,archived,author,author_created_utc,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,removal_reason,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,author_cakeday
0,False,ccmcg3,1.488827e+09,None,None,[],None,None,None,text,...,None,1544848333,10,True,False,relationships,t5_2qjvn,r/relationships,public,NaN
1,False,prolific_hype,1.517951e+09,None,None,[],None,None,None,text,...,None,1544848334,5,True,False,relationships,t5_2qjvn,r/relationships,public,NaN
2,False,AutoModerator,1.325741e+09,None,None,[],None,None,None,text,...,None,1544848336,1,False,False,relationships,t5_2qjvn,r/relationships,public,NaN
3,False,Temprament,1.365130e+09,None,None,[],None,None,None,text,...,None,1544848336,79,True,False,relationships,t5_2qjvn,r/relationships,public,NaN
4,False,AutoModerator,1.325741e+09,None,None,[],None,None,None,text,...,None,1544848342,1,False,False,relationships,t5_2qjvn,r/relationships,public,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262258,False,Somethoughts44,1.473577e+09,None,None,[],None,None,None,text,...,None,1546258231,2,True,False,relationships,t5_2qjvn,r/relationships,public,NaN
262259,False,Redz1990,1.462126e+09,None,None,[],None,None,None,text,...,None,1546258233,1,True,False,relationships,t5_2qjvn,r/relationships,public,NaN
262260,False,[deleted],NaN,,None,NaN,None,None,dark,NaN,...,None,1546258235,1,True,False,relationships,t5_2qjvn,r/relationships,public,NaN
262261,False,Levelfivemage,1.518480e+09,None,None,[],None,None,None,text,...,None,1546258251,1,True,False,relationships,t5_2qjvn,r/relationships,public,NaN


In [10]:
# this converts a zst file to csv
#
# it's important to note that the resulting file will likely be quite large
# and you probably won't be able to open it in excel or another csv reader
#
# arguments are inputfile, outputfile, fields
# call this like
# python to_csv.py wallstreetbets_submissions.zst wallstreetbets_submissions.csv author,selftext,title

import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers
import pandas as pd


log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()


if __name__ == "__main__":
	input_file_path = "relationships_comments_new.zst"
	output_file_path = "relationships_comments_17_22.csv"
	fields = "author"

	file_size = os.stat(input_file_path).st_size
	file_lines = 0
	file_bytes_processed = 0
	line = None
	created = None
	bad_lines = 0
	output_file = open(output_file_path, "w", encoding='utf-8', newline="")
	writer = csv.writer(output_file)
	#writer.writerow(fields)
	rows=[]
	try:
		for line, file_bytes_processed in read_lines_zst(input_file_path):
			try:
				obj = json.loads(line)
				output_obj = []
				#print(obj)
				#rows.append(obj)
                
				#for field in fields:
					#output_obj.append(str(obj[field]).encode("utf-8", errors='replace').decode())
				#writer.writerow(output_obj)
#first date is end date und last date is first date
				created = datetime.utcfromtimestamp(int(obj['created_utc']))
				if created.strftime('%Y-%m-%d %H:%M:%S') < "2019-11-30 23:59:59" and created.strftime('%Y-%m-%d %H:%M:%S')  > "2019-11-01 00:00:00":
					rows.append(obj)
			except json.JSONDecodeError as err:
				bad_lines += 1
			file_lines += 1
			#if file_lines % 100000 == 0:
				#log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
	except KeyError as err:
		log.info(f"Object has no key: {err}")
		log.info(line)
	except Exception as err:
		log.info(err)
		log.info(line)

	output_file.close()
	log.info(f"Complete : {file_lines:,} : {bad_lines:,}")
	df_4=pd.DataFrame.from_dict(rows, orient='columns')

Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0


In [11]:
df_4

,all_awardings,associated_award,author,author_created_utc,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,...,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,total_awards_received,author_cakeday
0,[],None,desaparecidose,1.400310e+09,None,None,[],None,None,None,...,1580511454,3,True,False,relationships,t5_2qjvn,r/relationships,public,0,NaN
1,[],None,mainshkweez,NaN,None,None,[],None,None,None,...,1580511499,1,True,False,relationships,t5_2qjvn,r/relationships,public,0,NaN
2,[],None,Roccet_MS,1.487616e+09,None,None,[],None,None,None,...,1580511502,3,True,False,relationships,t5_2qjvn,r/relationships,public,0,NaN
3,[],None,stephowl,NaN,None,None,[],None,None,None,...,1580511502,4,True,False,relationships,t5_2qjvn,r/relationships,public,0,NaN
4,[],None,AMerrickanGirl,1.171318e+09,None,None,[],None,None,None,...,1580511531,1,True,False,relationships,t5_2qjvn,r/relationships,public,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198320,[],None,[deleted],NaN,,None,NaN,None,None,dark,...,1584150565,1,True,False,relationships,t5_2qjvn,r/relationships,public,0,NaN
198321,[],None,danx1000,1.346275e+09,None,None,[],None,None,None,...,1584150584,1,True,False,relationships,t5_2qjvn,r/relationships,public,0,NaN
198322,[],None,ErgonomicCat,1.428530e+09,None,None,[],None,None,None,...,1584150584,2,True,False,relationships,t5_2qjvn,r/relationships,public,0,NaN
198323,[],None,thewordofrob,1.455036e+09,None,None,[],None,None,None,...,1584150607,2,True,False,relationships,t5_2qjvn,r/relationships,public,0,NaN


In [12]:
df_4.columns

Index(['all_awardings', 'associated_award', 'author', 'author_created_utc',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_template_id',
       'author_flair_text', 'author_flair_text_color', 'author_flair_type',
       'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders',
       'body', 'can_gild', 'can_mod_post', 'collapsed',
       'collapsed_because_crowd_control', 'collapsed_reason',
       'controversiality', 'created_utc', 'distinguished', 'edited', 'gilded',
       'gildings', 'id', 'is_submitter', 'link_id', 'locked', 'no_follow',
       'parent_id', 'permalink', 'quarantined', 'removal_reason',
       'retrieved_on', 'score', 'send_replies', 'stickied', 'subreddit',
       'subreddit_id', 'subreddit_name_prefixed', 'subreddit_type',
       'total_awards_received', 'author_cakeday'],
      dtype='object')

In [13]:
# this converts a zst file to csv
#
# it's important to note that the resulting file will likely be quite large
# and you probably won't be able to open it in excel or another csv reader
#
# arguments are inputfile, outputfile, fields
# call this like
# python to_csv.py wallstreetbets_submissions.zst wallstreetbets_submissions.csv author,selftext,title

import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers
import pandas as pd


log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()


if __name__ == "__main__":
	input_file_path = "relationships_comments_new.zst"
	output_file_path = "relationships_comments_17_22.csv"
	fields = "author"

	file_size = os.stat(input_file_path).st_size
	file_lines = 0
	file_bytes_processed = 0
	line = None
	created = None
	bad_lines = 0
	output_file = open(output_file_path, "w", encoding='utf-8', newline="")
	writer = csv.writer(output_file)
	#writer.writerow(fields)
	rows=[]
	try:
		for line, file_bytes_processed in read_lines_zst(input_file_path):
			try:
				obj = json.loads(line)
				output_obj = []
				#print(obj)
				#rows.append(obj)
                
				#for field in fields:
					#output_obj.append(str(obj[field]).encode("utf-8", errors='replace').decode())
				#writer.writerow(output_obj)
#first date is end date und last date is first date
				created = datetime.utcfromtimestamp(int(obj['created_utc']))
				if created.strftime('%Y-%m-%d %H:%M:%S') < "2020-11-30 23:59:59" and created.strftime('%Y-%m-%d %H:%M:%S')  > "2020-11-01 00:00:00":
					rows.append(obj)
			except json.JSONDecodeError as err:
				bad_lines += 1
			file_lines += 1
			#if file_lines % 100000 == 0:
				#log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
	except KeyError as err:
		log.info(f"Object has no key: {err}")
		log.info(line)
	except Exception as err:
		log.info(err)
		log.info(line)

	output_file.close()
	log.info(f"Complete : {file_lines:,} : {bad_lines:,}")
	df_5=pd.DataFrame.from_dict(rows, orient='columns')

Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0


In [14]:
df_5

,all_awardings,associated_award,author,author_created_utc,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,...,send_replies,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,top_awarded_type,total_awards_received,treatment_tags,author_cakeday
0,[],None,AutoModerator,1.325741e+09,None,None,[],None,None,None,...,False,False,relationships,t5_2qjvn,r/relationships,public,None,0,[],NaN
1,[],None,Surroundedbymor0ns,NaN,None,None,[],None,None,None,...,True,False,relationships,t5_2qjvn,r/relationships,public,None,0,[],NaN
2,[],None,readytobinformed247,1.548501e+09,None,None,[],None,None,None,...,True,False,relationships,t5_2qjvn,r/relationships,public,None,0,[],NaN
3,[],None,fightmaxmaster,1.461546e+09,None,None,[],None,None,None,...,True,False,relationships,t5_2qjvn,r/relationships,public,None,0,[],NaN
4,[],None,yoga1313,NaN,None,None,[],None,None,None,...,True,False,relationships,t5_2qjvn,r/relationships,public,None,0,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137211,[],None,MonsterFactoryX,NaN,None,None,[],None,None,None,...,True,False,relationships,t5_2qjvn,r/relationships,public,None,0,[],NaN
137212,[],None,Jay_Edgar,NaN,None,None,[],None,None,None,...,True,False,relationships,t5_2qjvn,r/relationships,public,None,0,[],NaN
137213,[],None,Substantial-Two2941,NaN,None,None,[],None,None,None,...,True,False,relationships,t5_2qjvn,r/relationships,public,None,0,[],NaN
137214,[],None,shmookieguinz,1.420486e+09,None,None,[],None,None,None,...,True,False,relationships,t5_2qjvn,r/relationships,public,None,0,[],NaN


In [15]:
df_5.columns

Index(['all_awardings', 'associated_award', 'author', 'author_created_utc',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_template_id',
       'author_flair_text', 'author_flair_text_color', 'author_flair_type',
       'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders',
       'body', 'can_gild', 'can_mod_post', 'collapsed',
       'collapsed_because_crowd_control', 'collapsed_reason', 'comment_type',
       'controversiality', 'created_utc', 'distinguished', 'edited', 'gilded',
       'gildings', 'id', 'is_submitter', 'link_id', 'locked', 'no_follow',
       'parent_id', 'permalink', 'quarantined', 'removal_reason',
       'retrieved_on', 'score', 'send_replies', 'stickied', 'subreddit',
       'subreddit_id', 'subreddit_name_prefixed', 'subreddit_type',
       'top_awarded_type', 'total_awards_received', 'treatment_tags',
       'author_cakeday'],
      dtype='object')

In [16]:
# this converts a zst file to csv
#
# it's important to note that the resulting file will likely be quite large
# and you probably won't be able to open it in excel or another csv reader
#
# arguments are inputfile, outputfile, fields
# call this like
# python to_csv.py wallstreetbets_submissions.zst wallstreetbets_submissions.csv author,selftext,title

import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers
import pandas as pd


log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()


if __name__ == "__main__":
	input_file_path = "relationships_comments_new.zst"
	output_file_path = "relationships_comments_17_22.csv"
	fields = "author"

	file_size = os.stat(input_file_path).st_size
	file_lines = 0
	file_bytes_processed = 0
	line = None
	created = None
	bad_lines = 0
	output_file = open(output_file_path, "w", encoding='utf-8', newline="")
	writer = csv.writer(output_file)
	#writer.writerow(fields)
	rows=[]
	try:
		for line, file_bytes_processed in read_lines_zst(input_file_path):
			try:
				obj = json.loads(line)
				output_obj = []
				#print(obj)
				#rows.append(obj)
                
				#for field in fields:
					#output_obj.append(str(obj[field]).encode("utf-8", errors='replace').decode())
				#writer.writerow(output_obj)
#first date is end date und last date is first date
				created = datetime.utcfromtimestamp(int(obj['created_utc']))
				if created.strftime('%Y-%m-%d %H:%M:%S') < "2021-11-30 23:59:59" and created.strftime('%Y-%m-%d %H:%M:%S')  > "2021-11-01 00:00:00":
					rows.append(obj)
			except json.JSONDecodeError as err:
				bad_lines += 1
			file_lines += 1
			#if file_lines % 100000 == 0:
				#log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
	except KeyError as err:
		log.info(f"Object has no key: {err}")
		log.info(line)
	except Exception as err:
		log.info(err)
		log.info(line)

	output_file.close()
	log.info(f"Complete : {file_lines:,} : {bad_lines:,}")
	df_6=pd.DataFrame.from_dict(rows, orient='columns')

Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0


In [17]:
df_6

,all_awardings,archived,associated_award,author,author_created_utc,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,...,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,top_awarded_type,total_awards_received,treatment_tags,unrepliable_reason,author_cakeday,editable
0,[],False,None,Admirable_Share_5843,1.624339e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
1,[],False,None,[deleted],NaN,,None,NaN,None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
2,[],False,None,dancing_chinese_kid,1.475164e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
3,[],False,None,AutoModerator,1.325741e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
4,[],False,None,excodaIT,1.612285e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150815,[],False,None,Difficult_Buffalo327,1.622590e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
150816,[],False,None,Darknatio,1.631227e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
150817,[],False,None,AutoModerator,1.325741e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
150818,[],False,None,bilbosdottir,1.548805e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN


In [18]:
df_6.columns

Index(['all_awardings', 'archived', 'associated_award', 'author',
       'author_created_utc', 'author_flair_background_color',
       'author_flair_css_class', 'author_flair_richtext',
       'author_flair_template_id', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'author_fullname',
       'author_patreon_flair', 'author_premium', 'body', 'can_gild',
       'collapsed', 'collapsed_because_crowd_control', 'collapsed_reason',
       'collapsed_reason_code', 'comment_type', 'controversiality',
       'created_utc', 'distinguished', 'edited', 'gilded', 'gildings', 'id',
       'is_submitter', 'link_id', 'locked', 'name', 'no_follow', 'parent_id',
       'permalink', 'retrieved_on', 'score', 'score_hidden', 'send_replies',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_name_prefixed',
       'subreddit_type', 'top_awarded_type', 'total_awards_received',
       'treatment_tags', 'unrepliable_reason', 'author_cakeday', 'editable'],
      dtype='obje

In [19]:
# this converts a zst file to csv
#
# it's important to note that the resulting file will likely be quite large
# and you probably won't be able to open it in excel or another csv reader
#
# arguments are inputfile, outputfile, fields
# call this like
# python to_csv.py wallstreetbets_submissions.zst wallstreetbets_submissions.csv author,selftext,title

import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers
import pandas as pd


log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()


if __name__ == "__main__":
	input_file_path = "relationships_comments_new.zst"
	output_file_path = "relationships_comments_17_22.csv"
	fields = "author"

	file_size = os.stat(input_file_path).st_size
	file_lines = 0
	file_bytes_processed = 0
	line = None
	created = None
	bad_lines = 0
	output_file = open(output_file_path, "w", encoding='utf-8', newline="")
	writer = csv.writer(output_file)
	#writer.writerow(fields)
	rows=[]
	try:
		for line, file_bytes_processed in read_lines_zst(input_file_path):
			try:
				obj = json.loads(line)
				output_obj = []
				#print(obj)
				#rows.append(obj)
                
				#for field in fields:
					#output_obj.append(str(obj[field]).encode("utf-8", errors='replace').decode())
				#writer.writerow(output_obj)
#first date is end date und last date is first date
				created = datetime.utcfromtimestamp(int(obj['created_utc']))
				if created.strftime('%Y-%m-%d %H:%M:%S') < "2022-11-30 23:59:59" and created.strftime('%Y-%m-%d %H:%M:%S')  > "2022-11-01 00:00:00":
					rows.append(obj)
			except json.JSONDecodeError as err:
				bad_lines += 1
			file_lines += 1
			#if file_lines % 100000 == 0:
				#log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
	except KeyError as err:
		log.info(f"Object has no key: {err}")
		log.info(line)
	except Exception as err:
		log.info(err)
		log.info(line)

	output_file.close()
	log.info(f"Complete : {file_lines:,} : {bad_lines:,}")
	df_7=pd.DataFrame.from_dict(rows, orient='columns')

Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0
Complete : 26,908,630 : 0


In [20]:
df_7

,all_awardings,archived,associated_award,author,author_created_utc,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,...,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,top_awarded_type,total_awards_received,treatment_tags,unrepliable_reason,author_cakeday,editable
0,[],False,None,Archer2795,1.581028e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
1,[],False,None,floridorito,1.590522e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
2,[],False,None,testingtesting86,1.650500e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
3,[],False,None,NerdYogi,1.512931e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
4,[],False,None,[deleted],NaN,,None,NaN,None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145572,[],False,None,Wellwellwelllooky,1.657479e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
145573,[],False,None,Flower-of-Telperion,1.551310e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
145574,[],False,None,Tequila_Heineken,1.505648e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN
145575,[],False,None,103_with_reddit_ref,1.644850e+09,None,None,[],None,None,...,relationships,t5_2qjvn,r/relationships,public,None,0,[],None,NaN,NaN


In [21]:
df_7.columns

Index(['all_awardings', 'archived', 'associated_award', 'author',
       'author_created_utc', 'author_flair_background_color',
       'author_flair_css_class', 'author_flair_richtext',
       'author_flair_template_id', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'author_fullname',
       'author_patreon_flair', 'author_premium', 'body', 'can_gild',
       'collapsed', 'collapsed_because_crowd_control', 'collapsed_reason',
       'collapsed_reason_code', 'comment_type', 'controversiality',
       'created_utc', 'distinguished', 'edited', 'gilded', 'gildings', 'id',
       'is_submitter', 'link_id', 'locked', 'name', 'no_follow', 'parent_id',
       'permalink', 'retrieved_on', 'score', 'score_hidden', 'send_replies',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_name_prefixed',
       'subreddit_type', 'top_awarded_type', 'total_awards_received',
       'treatment_tags', 'unrepliable_reason', 'author_cakeday', 'editable'],
      dtype='obje

Save CSV

In [22]:
df.to_csv("relationship_comments_16_new.csv",sep=";")
df_2.to_csv("relationship_comments_17_new.csv",sep=";")
df_3.to_csv("relationship_comments_18_new.csv",sep=";")
df_4.to_csv("relationship_comments_19_new.csv",sep=";")
df_5.to_csv("relationship_comments_20_new.csv",sep=";")
df_6.to_csv("relationship_comments_21_new.csv",sep=";")
df_7.to_csv("relationship_comments_22_new.csv",sep=";")

In [23]:
ls

Final scrap.ipynb                 relationship_comments_20.csv
Final scrap_new.ipynb             relationship_comments_20_new.csv
relationship_comments_16.csv      relationship_comments_21.csv
relationship_comments_16_new.csv  relationship_comments_21_new.csv
relationship_comments_17.csv      relationship_comments_22.csv
relationship_comments_17_new.csv  relationship_comments_22_new.csv
relationship_comments_18.csv      relationships_comments.zst
relationship_comments_18_new.csv  relationships_comments_17_22.csv
relationship_comments_19.csv      relationships_comments_new.zst
relationship_comments_19_new.csv
